Search Engine with Tools and Agents

In [1]:
## Tool creation for in-built tools
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv)

api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)


In [3]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max= 250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
tools = [arxiv, wiki]

In [5]:
# Custom Tools [RAG Tools]
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
import os
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250).split_documents(docs)
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=openai_api_key))
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x736f8ea65330>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search", "Search any information about the langsmith ")
retriever_tool.name

'langsmith-search'

In [10]:
tools = [wiki, arxiv, retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/devendra/Desktop/Udemy_Generative_AI_With_Langchain_and_Huggingface_2024/Project/Langchain_Projects/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about the langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x736f8ea73be0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorst

In [15]:
## Run all the tools with Agents and LLM
# Tools, LLM --> AgentExecutor

from langchain_groq import ChatGroq

groq_api_key= os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name = "Llama3-8b-8192")


In [16]:
# prompt template 
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
## Agents 
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [18]:
from langchain.agents import AgentExecutor
agent_executor =  AgentExecutor(agent= agent, tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_executor.invoke({"input":"Tell me about the langsmith ?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get 

{'input': 'Tell me about the langsmith ?',
 'output': "It seems like you're looking for information about LangSmith. LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features such as observability, evals, and prompt engineering to help developers build, test, and optimize their AI applications.\n\nWith LangSmith, you can closely monitor and evaluate your application, making it easier to ship quickly and with confidence. The platform offers observability features that allow you to analyze traces and configure metrics, dashboards, and alerts. You can also use LangSmith to evaluate your application over production traffic, score performance, and get human feedback on your data.\n\nAdditionally, LangSmith provides tools for prompt engineering, which enables you to iterate on prompts with automatic version control and collaboration features. LangSmith is framework-agnostic, meaning it can be used with or without LangChain's open-sour

In [ ]:
agent_executor.invoke({"input":"What is machine learning ?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result from the tool call, I can continue to provide more information about machine learning.

Machine learning is a subset of artificial intelligence that involves training algorithms to learn from data and improve their performance over time. It is a type of supervised learning that involves training a model on labeled data, so that it can make predictions or take actions based on new, unseen data.

There are many applications of machine learning, including image and speech recognition, natural language processing, recommender systems, and predictive analytics. It is widely used in many industries, including healthcare, finan

{'input': 'What is machine learning ?',
 'output': 'Based on the result from the tool call, I can continue to provide more information about machine learning.\n\nMachine learning is a subset of artificial intelligence that involves training algorithms to learn from data and improve their performance over time. It is a type of supervised learning that involves training a model on labeled data, so that it can make predictions or take actions based on new, unseen data.\n\nThere are many applications of machine learning, including image and speech recognition, natural language processing, recommender systems, and predictive analytics. It is widely used in many industries, including healthcare, finance, marketing, and more.\n\nWould you like to know more about a specific aspect of machine learning, or would you like to explore a different topic?'}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1706.08386 about ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.08386'}`


Published: 2017-06-23
Title: Flows, scaling, and the control of moment hierarchies for stochastic chemical reaction networks
Authors: Eric Smith, Supriya Krishnamurthy
Summary: Stochastic chemical reaction networks (CRNs) are complex systems which
coThe paper "Flows, scaling, and the control of moment hierarchies for stochastic chemical reaction networks" by Eric Smith and Supriya Krishnamurthy, published in 2017, discusses stochastic chemical reaction networks (CRNs) and their complexity.

> Finished chain.


{'input': "What's the paper 1706.08386 about ?",
 'output': 'The paper "Flows, scaling, and the control of moment hierarchies for stochastic chemical reaction networks" by Eric Smith and Supriya Krishnamurthy, published in 2017, discusses stochastic chemical reaction networks (CRNs) and their complexity.'}